In [160]:
#1 - save newline and space info (DONE, See below)
#  a. - should this be in the tokenizer decode function
#  b. - Did we need to add something to the vocab??
#2 - add delineating NL between snippets (Done)
#  a. - This is taken care of when we generate the aligned snippets
#  b. - The need for a delineating new line seems like a drawback, any way to do it without one?
#3 - generate aligned snippets (Moderate success, works most of the time it appears)
#  a. - This comes down to making the sure the first token(s) of the samples are the same
#  b. - for now this will be only the first token, but you could do it for sever, it works but much fewer have the same X-number of tokens, sometimes gets similar functions,
#  c. - there are cases this doesn't cover, such as a class that has defined functions of its own (i.e., naturally occurring diff alignment)
#  d. - takes longer than I expected??
#  e. - making more than 1 alignment token can make it process a lot, sometimes get similar code interestingly
#4 - create the data for LSTM and save it (1000)
#  a. - initial param: window diameter 20
#  b. - inconsistent number of items per window!?

#other:
#-------------------------------------------------
#language related things, importance of whitespace
#https://en.wikipedia.org/wiki/Whitespace_(programming_language)
#https://en.wikipedia.org/wiki/Brainfuck

#what about size of input for training the tokenizer

#issue with ▁ token

#are AST tokens akin to named entity recognition or semantic role labeling... i guess it would be syntactic role labeling?

In [1]:
from csv import reader

In [2]:
# open file in read mode
number_of_lines = 0
num_chars = 0 #TODO
with open('code-comment-300-300.csv', 'r') as read_obj:
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    # Iterate over each row in the csv using reader object
    
    data = []
    for row in csv_reader:
        # row variable is a list that represents a row in csv
        #row[0] - code
        #row[1] - comment
        
        number_of_lines+=row[0].count('\n')
        
        #way to handle white spaces:
        #space #do first, 
        new_code = row[0].replace(' ',' SPACE')
        #newline
        new_code = new_code.replace('\n',' NEWLINE')
        #tab
        new_code = new_code.replace('\t',' TAB')
        
        #TODO
        #FILTER FOR UNICODE THING
        
        
        #save new data
        row = [new_code,row[1]]
        data.append(row)
    
    read_obj.close()
    
print('num samples: ', len(data))
print('num lines: ', number_of_lines)

num samples:  1278738
num lines:  7729264


In [3]:
#https://pypi.org/project/sentencepiece/
#http://ethen8181.github.io/machine-learning/deep_learning/subword/bpe.html

In [4]:
with open('data.txt', 'w') as filehandle:
    for i in range(len(data)):
        filehandle.write('%s\n' % data[i][0])

In [3]:
import sentencepiece as spm

In [4]:
class Tokenizer:

    def __init__(self, filepath='python_tokenizer_30k.model'):
        self.sp = spm.SentencePieceProcessor(model_file=filepath)

    def encode(self, text, t=int):
        return self.sp.encode(text, out_type=t)

    def decode(self, pieces):
        return self.sp.decode(pieces)

    @staticmethod
    def train(input_file='data/raw_sents.txt', model_prefix='sp_model', vocab_size=30522):
        spm.SentencePieceTrainer.train(input=input_file, model_prefix=model_prefix, vocab_size=vocab_size,
                                       #input_sentence_size=2 ** 16, shuffle_input_sentence=True)
                                       input_sentence_size=number_of_lines, shuffle_input_sentence=True)

In [8]:
#train
Tokenizer.train(input_file='data.txt', model_prefix='python_tokenizer_30k', vocab_size=30000) #model_prefix is model storage name

In [5]:
#instantiate tokenizer model
tokenizer = Tokenizer('python_tokenizer.model')

In [6]:
#see some tokenized data (code)
data[1][0]

' SPACE SPACE SPACE SPACEdef SPACEunlock_and_close(self): NEWLINE SPACE SPACE SPACE SPACE SPACE SPACE NEWLINE SPACE SPACE SPACE SPACE SPACE SPACEif SPACEself._locked: NEWLINE SPACE SPACE SPACE SPACE SPACE SPACE SPACE SPACEfcntl.lockf(self._fh.fileno(), SPACEfcntl.LOCK_UN) NEWLINE SPACE SPACE SPACE SPACE SPACE SPACEself._locked SPACE= SPACEFalse NEWLINE SPACE SPACE SPACE SPACE SPACE SPACEif SPACEself._fh: NEWLINE SPACE SPACE SPACE SPACE SPACE SPACE SPACE SPACEself._fh.close() NEWLINE'

In [7]:
#tokenize code (ie encode)
#with words
#tokens = tokenizer.encode(data[1][0],t=str)
#with numbers
tokens = tokenizer.encode(data[1][0])
print(tokens)

[3, 3, 3, 13, 3, 3565, 5, 303, 5, 255, 6, 8, 10, 4, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 24, 3, 8, 21, 2328, 15, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4818, 7, 529, 73, 6, 8, 21, 2040, 7, 3658, 336, 3, 4818, 7, 4693, 5, 1826, 12, 4, 3, 3, 3, 3, 3, 3, 8, 21, 2328, 3, 11, 3, 72, 4, 3, 3, 3, 3, 3, 24, 3, 8, 21, 2040, 15, 4, 3, 3, 3, 3, 3, 3, 3, 3, 8, 21, 2040, 7, 255, 19, 4]


In [8]:
#detokenize
def decode_tokenized_code_snippet(tokens):
    decoded = tokenizer.decode(tokens)
    #decode but still has the added strings
    #print(decoded)
    token_string = ''.join(decoded)
    token_string = token_string.replace('SPACE',' ')
    token_string = token_string.replace('NEWLINE','\n')
    token_string = token_string.replace('TAB','\t')
    #print(token_string)
    return token_string

print(decode_tokenized_code_snippet(tokens))

       def  unlock_and_close(self): 
             
            if  self._locked: 
                fcntl.lockf(self._fh.fileno(),  fcntl.LOCK_UN) 
            self._locked  =  False 
            if  self._fh: 
                self._fh.close() 



In [9]:
import random
from random import sample
random.seed(10)

In [11]:
#[depricated]: see generate_verified_code_from_data  
def generate_code_from_data(data,num_samples):
    # 
    code_indixes = list(range(1,len(data)))
    sampled_indexes = sample(code_indixes,num_samples)

    generated_code = []
    generated_code_ids = []
    break_locations = []
    for i in sampled_indexes:
        
        #get raw code
        sampled_code_snippet = data[i][0] #just the code, thus the 0
        
        #tokenize it 
        #with words
        sampled_code_snippet = tokenizer.encode(data[i][0],t=str)
        #with numbers
        sampled_code_snippet_ids = tokenizer.encode(data[i][0])
        
        #first break
        if len(break_locations)==0:
            break_locations.append(len(sampled_code_snippet))
        
        #add length from the previous one
        else:
            break_locations.append(len(sampled_code_snippet)+break_locations[-1])
            
        #concantonate the code sample
        generated_code+=sampled_code_snippet
        generated_code_ids+=sampled_code_snippet_ids
    
    return generated_code, break_locations, generated_code_ids

In [ ]:
'''
code, breaks, _ = generate_code_from_data(data,3)
#do it with the code tokens
wd=20 #window diameter
X_windows = centered_sliding_window(code,wd,encode=True)
#print(X_windows)
'''

In [16]:
#[depricated]: see generate_verified_code_from_data  

#note: it seems "harder" to generate these aligned snippets than I expected
def generate_aligned_code_from_data(data,num_samples):
    
    #get all indixes of code
    code_indixes = list(range(1,len(data)))
    
    tokenized_code = []
    #generated_code_ids = []
    break_locations = []
    
    #pick a random snippet
    first_index = random.choice(code_indixes)
    first_sampled_code_snippet = data[first_index][0] #just the code, thus the 0
    
    #tokenize it 
    #with words
    #tokenized_code_snippet = tokenizer.encode(data[i][0],t=str)
    tokenized_code_snippet = tokenizer.encode(first_sampled_code_snippet,t=str)
    #with numbers
    #sampled_code_snippet_ids = tokenizer.encode(data[i][0])
    
    #THIS IS HOW WE ALIGN IT??? CAUSE WHY?
    #FIRST TOKEN
    #just the first one
    alignment_token = tokenized_code_snippet[0]
    #first x
    #x=2
    #alignment_token = tokenized_code_snippet[:x]
    
    #save first snippet to the generated tokenized code
    tokenized_code+=tokenized_code_snippet
    
    #pick more random snippets until we find two more with same first token
    #find other snippets with same first alingment 
    while len(break_locations)<num_samples-1:
        
        #get new random candidate
        new_index = random.choice(code_indixes)
        candidate_next_snippet = data[new_index][0]
        
        #do i need to make sure it doesnt repeat itself??
        
        #tokenize it 
        #with words
        tokenized_candidate = tokenizer.encode(candidate_next_snippet,t=str)
        if tokenized_candidate[0]==alignment_token:
        #if tokenized_candidate[:x]==alignment_token:
            
            #add delinating token before adding new snippet tokens
            #If there is no new line at end of snippet
            if tokenized_code[-1][-7:]!='NEWLINE':
                tokenized_code+=[' NEWLINE']
            #save break location
            break_locations.append(len(tokenized_code)-1)
            tokenized_code+=tokenized_candidate
            #print('good--')
            #-1 right?
            
        #not alinged
        else:
            pass
            #print('bad?')
            
    return tokenized_code, break_locations
        
    
#generate_aligned_code_from_data(data,3)

In [ ]:
'''
code, breaks = generate_aligned_code_from_data(data,3)
#do it with the code tokens
wd=20 #window diameter
X_windows = centered_sliding_window(code,wd,encode=True)
#print(X_windows)
'''

In [20]:
#Alignment Utils for generate_aligned_code_from_data_2 [Depricated]

#find the number of those tokens at the begining that goes uninterupted
def find_alignment_series(l):
    #find first token
    first_token = l[0]
    
    #find how many of these tokens are a the begining
    num = 1
    uninterupted=True
    while uninterupted == True:
        if l[num]==first_token:
            num+=1
        else:
            uninterupted=False
    return [first_token]*num
    
#additional snippets are aligned if they also have that many uninterupted tokens
def check_alignment(a,b):
    return len(find_alignment_series(a)) == len(find_alignment_series(b))
    
    
a = ['a','a','a','b','a','b','c'] #first list
#alignment should be ['a','a','a']

b = ['a','b','a','b','c'] #not aligned
c = ['a','a','a','c']     #aligned

alignment_series = find_alignment_series(a)
print(check_alignment(a,b))
print(check_alignment(a,c))


False
True


In [21]:
#[depricated]: see generate_verified_code_from_data  

#Alignment handeled by generic string analizer
def generate_aligned_code_from_data_2(data,num_samples):
    
    #get all indixes of code
    code_indixes = list(range(1,len(data)))
    
    tokenized_code = []
    break_locations = []
    
    #pick a random snippet
    first_index = random.choice(code_indixes)
    first_sampled_code_snippet = data[first_index][0] #just the code, thus the 0
    first_tokenized_code_snippet = tokenizer.encode(first_sampled_code_snippet,t=str)
    tokenized_code+=first_tokenized_code_snippet
    
    #print('original:',find_alignment_series(first_tokenized_code_snippet))
    
    #pick more random snippets until we find two more with same first token
    #find other snippets with same first alingment 
    while len(break_locations)<num_samples-1:
        
        #get new random candidate
        ###do i need to make sure it doesnt repeat itself??
        new_index = random.choice(code_indixes)
        candidate_next_snippet = data[new_index][0]
        #tokenize it with words
        tokenized_candidate = tokenizer.encode(candidate_next_snippet,t=str)
        #print(find_alignment_series(tokenized_candidate))
        
        if check_alignment(first_tokenized_code_snippet,tokenized_candidate):
            #print(find_alignment_series(tokenized_candidate))
            
            #add delinating token before adding new snippet tokens
            #If there is no new line at end of snippet
            if tokenized_code[-1][-7:]!='NEWLINE':
                tokenized_code+=[' NEWLINE']
            
            #save break location
            break_locations.append(len(tokenized_code)-1)
            tokenized_code+=tokenized_candidate
            #print('good')
            
        #not alinged
        else:
            pass
            #print('bad?')
            
    return tokenized_code, break_locations
        
    
#generate_aligned_code_from_data_2(data,3)

In [16]:
'''
code, breaks = generate_aligned_code_from_data_2(data,3)
#do it with the code tokens
wd=20 #window diameter
X_windows = centered_sliding_window(code,wd,encode=True)
#print(X_windows)
'''

In [10]:
########
#from Tokenize_python
#######
import ast
import tokenize
import io
import numpy as np
from tqdm import tqdm #inline progress bar (quality of life)
import collections

In [11]:
def check_code_validity(code_snippet):
    valid=False
    try:
        code_snippet = decode_tokenized_code_snippet(code_snippet)
        tokens =  tokenize.tokenize(io.BytesIO(code_snippet.encode('utf-8')).readline)
        for t in tokens:
            #this is enough to trigger it
            pass
        valid=True
    except Exception as e:
        pass
        #print(e)
    return valid
    

#prints percent valid snippets in data
'''
v=0
nv=0
for d in data:
    snip = d[0]
    if check_code_validity(snip):
        v+=1
    else:
        nv+=1
print((v/(v+nv))*100)
'''

'\nv=0\nnv=0\nfor d in data:\n    snip = d[0]\n    if check_code_validity(snip):\n        v+=1\n    else:\n        nv+=1\nprint((v/(v+nv))*100)\n'

In [12]:
def get_random_tokenized_snippet(data,code_indixes):
    i = random.choice(code_indixes)
    snip = data[i][0] #just the code, thus the 0
    tokenized_snip =  tokenizer.encode(snip,t=str)
    return tokenized_snip

In [13]:
#alinging by validing with AST
def generate_verified_code_from_data(data,num_samples):
    
    #get all indixes of code
    code_indixes = list(range(1,len(data)))
    
    #prep data
    tokenized_code = []
    break_locations = []
    
    #valid segment search params
    trys = 0
    max_trys = 100
    
    
    #pick more random snippets until we find two more with same first token
    #find other snippets with same first alingment 
    while len(break_locations)<num_samples:
        
        
        #print('----- inside while, before getting new cadidate code')
        #print(decode_tokenized_code_snippet(tokenizer.decode(tokenized_code)))
        #input()
        
        #get new random candidate
        ###do i need to make sure it doesnt repeat itself??
        new_index = random.choice(code_indixes)
        candidate_next_snippet = data[new_index][0]
        #tokenize it with words
        tokenized_candidate=[]
        tokenized_candidate = tokenizer.encode(candidate_next_snippet,t=str)
        #print(find_alignment_series(tokenized_candidate))
        
        #print('----- got tokenized candidate code')
        #print(decode_tokenized_code_snippet(tokenizer.decode(tokenized_code)))
        #input()
        
        
        #if temp_tokenized_code[-1][-7:]!='NEWLINE':
        #    temp_tokenized_code+=['NEWLINE']
        if len(tokenized_code)>1 and tokenized_code[-1][-8:]!='▁NEWLINE':
            #print('here')
            tokenized_code+=['▁NEWLINE']
        
        temp_tokenized_code = tokenized_code + tokenized_candidate
        
        #print('----- made the temporaty tokenized code to check')
        #print(decode_tokenized_code_snippet(tokenizer.decode(tokenized_code)))
        #input()
        
        #see if it is aligned (ie a valid code snippet)
        if check_code_validity(temp_tokenized_code):
            #print('good')
            tokenized_code=temp_tokenized_code
            break_locations.append(len(tokenized_code)-1)
                
            #print('----- just added new peice ')
            #print(decode_tokenized_code_snippet(tokenizer.decode(tokenized_code)))
            
                  
        else:
            #reset the test snippet
            temp_tokenized_code=[]
            trys+=1
            #print('bad',trys)
            #print(len(temp_tokenized_code))
        #print('-----')
        #print(decode_tokenized_code_snippet(tokenizer.decode(tokenized_code)))
        #print(len(tokenized_code))
        #input()
        
        
        #if we our snippet is hard to pair
        if trys>=max_trys:
            tokenized_code=[]
            break_locations=[]
            trys=0
            #print('restart')
    
    #we dont want to include the last break location, only ones between code segments
    return tokenized_code, break_locations[:-1]

#generate_aligned_code_from_data_3(data, 3)

In [14]:
#put comment in at break points
#https://www.tutorialspoint.com/python/list_insert.htm
#list.insert(index, obj)
def insert_comments(code, break_spots, comment='\n'+'*'*8+'\n',at_begining=True):
    #if there is a a comment at begining of snippet
    if at_begining:
        #adds a notation to add a 0
        #at beigning of break spots too
        break_spots.insert(0,0)
    
    
    #go through breaks backwards
    #so as not to mess up break 
    #spots as we would if we went forward
    for b in break_spots[::-1]:
        code.insert(b,comment)
    return code


#code=['a','b','c','d']
#c = insert_comments(code,[2])
#print(c)

In [15]:
####
####ALIGNMENT ISSUES, UNCOMMENT THE 'good' AND 'bad' PRINT,
####SEE SOEMTIMES WHEN UNICODE CHAR IS THERE IT MESSES UP
####


#code, breaks, _ = generate_code_from_data(data,3)
#code, breaks = generate_aligned_code_from_data_2(data,3)
code, breaks = generate_verified_code_from_data(data,3)
print('--------------------------------')
print('code:')
print('--------------------------------')
decoded = tokenizer.decode(code)
token_string = ''.join(decoded)
token_string = token_string.replace('SPACE',' ')
token_string = token_string.replace('NEWLINE','\n')
token_string = token_string.replace('TAB','\t')
print(token_string)
#print(code)
print('--------------------------------')
print('break spots: ',breaks)
print('--------------------------------')
print('code with break spots indicated:')
print('--------------------------------')
#insert_comments
comments_added = insert_comments(code,breaks)
comments_added_decoded = tokenizer.decode(comments_added)
comments_added_token_string = ''.join(comments_added_decoded)
comments_added_token_string = comments_added_token_string.replace('SPACE',' ')
comments_added_token_string = comments_added_token_string.replace('NEWLINE','\n')
comments_added_token_string = comments_added_token_string.replace('TAB','\t')
print(comments_added_token_string)
print('--------------------------------')

--------------------------------
code:
--------------------------------
       def  test_api_prefix(self): 
                 
                self.test_app( 
                        options={ 
                                'pyramid_jsonapi.route_pattern_api_prefix':  'api' 
                        }).get('/api/people') 
 
                self.direct_control  =  False 
                self.paths_to_delete  =  [] 

--------------------------------
break spots:  [108, 126]
--------------------------------
code with break spots indicated:
--------------------------------

********
        def  test_api_prefix(self): 
                 
                self.test_app( 
                        options={ 
                                'pyramid_jsonapi.route_pattern_api_prefix':  'api' 
                        }).get('/api/people') 

********
 
                self.direct_control  =  False
********
 
                self.paths_to_delete  =  [] 

--------------------------------


In [16]:
def centered_sliding_window(token_list, window_diamiter,encode=False,PAD='unk'):
    windows = []
    for i in range(len(token_list)):
        
        #print(token_list)
        #input()
        
        window = []
        
        #if we have to pad the begining
        if i < window_diamiter:
            before_len = window_diamiter-i
            before = [PAD]*before_len+token_list[0:i]
        else:
            before = token_list[i-window_diamiter:i]
        
        #if we have to pad the end
        if i+window_diamiter>=len(token_list):
            after_len = (i+1+window_diamiter)-len(token_list)
            after = token_list[i+1:i+1+window_diamiter]+[PAD]*after_len

        else:
            after = token_list[i+1:i+1+window_diamiter]
        
        #put it togeather
        #print('------')
        #print('before:',before)
        #print('center:',token_list[i])
        #print('after:',after)
        window = before + [token_list[i]] + after
        #for encoding code if we want
        if encode:
            new_window = []
            #print(window)
            #input()
            for i in window:
                encoded = tokenizer.encode(i)
                if len(encoded)>1:       
                    x=encoded[1]
                    if type(x)==list:
                        new_window.append(x[0])
                    else:
                        new_window.append(x)
                elif len(encoded)==1:
                    if type(encoded)==list:
                        new_window.append(encoded[0])
                    else:
                        new_window.append(encoded)
                else:
                    #for some reason it finds the unicode stuff __
                    pass
                    #print(window)
                    #print(i)
                    #print(encoded)
                    #input()
            #print(window)
            #print(len(window))
            #print(len(tokenizer.decode(window)))
            #print(tokenizer.decode(window))
            #print(len(tokenizer.encode(window)))
            #window = tokenizer.encode(tokenizer.decode(window))
            window = new_window
        #print(window)
        #print(len(window))
        #input()

        #save windowz
        windows.append(window)
    
    return windows



In [17]:
tokenizer.decode(201)

''

In [23]:
#example
wd=3#window diameter

l = ['a','b','c','d','e']#,'f','g','h','i','j','k','l']
windows = centered_sliding_window(l,wd,encode=False)
for w in windows:
    #print(len(w))
    print('center:',w[int(len(w)/2)],'- window: ',w)

center: a - window:  ['unk', 'unk', 'unk', 'a', 'b', 'c', 'd']
center: b - window:  ['unk', 'unk', 'a', 'b', 'c', 'd', 'e']
center: c - window:  ['unk', 'a', 'b', 'c', 'd', 'e', 'unk']
center: d - window:  ['a', 'b', 'c', 'd', 'e', 'unk', 'unk']
center: e - window:  ['b', 'c', 'd', 'e', 'unk', 'unk', 'unk']


In [111]:
def make_data_points():
    #code, breaks, _ = generate_code_from_data(data,3)
    #code, breaks = generate_aligned_code_from_data_2(data,3)
    code, breaks = generate_verified_code_from_data(data,3)
    
    #do it with the code tokens
    wd=20 #window diameter
    X_windows = centered_sliding_window(code,wd,encode=True)
    
    #do it with the ground truth 
    y = [0]*len(code)
    for b in breaks:
        y[b] = 1
    Y_windows = centered_sliding_window(y,wd,encode=False,PAD=0)
    #print(Y_windows)
    
    return X_windows, Y_windows


In [32]:
'''
#code, breaks = generate_aligned_code_from_data_2(data,3)
code, breaks = generate_verified_code_from_data(data,3)
y = [0]*len(code)
for b in breaks:
    y[b] = 1
Y_windows = centered_sliding_window(y,wd,encode=False, PAD=0)
#print(len(Y_windows))
#print(Y_windows)
'''

In [112]:
#SAVE DATA FOR BiLSTM

import json 

def make_generated_code_dataset(num_snippets):
    # Data to be written 
    LSMT_DATA = {}
    for i in tqdm(range(num_snippets)):
        x,y = make_data_points()
        LSMT_DATA[str(i)] = {'x':x,'y':y}

    with open("LSTM_DATA.json", "w") as outfile: 
        json.dump(LSMT_DATA, outfile)
        
make_generated_code_dataset(10000)

100%|██████████| 10000/10000 [14:00<00:00, 11.89it/s]
